In [1]:
import numpy as np
import pandas as pd
import konlpy
import matplotlib as plt
import re
from konlpy.tag import Mecab
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
ref=pd.read_csv("attraction_all_translate.csv",encoding='utf-8')
ref=ref.drop([ref.columns[6],ref.columns[7],ref.columns[8]],axis=1)
ref=ref.dropna(subset=['score'],axis=0)
ref.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27748 entries, 0 to 27751
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   space     27748 non-null  object 
 1   type      27748 non-null  object 
 2   language  27748 non-null  object 
 3   day       27748 non-null  int64  
 4   text      27748 non-null  object 
 5   score     27748 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 1.5+ MB


In [3]:
#language가 한국어가 아닐 수도 있는 경우 삭제
ref.drop(ref.loc[ref["language"]=="일본어"].index,inplace=True)
ref.drop(ref.loc[ref["language"]=="중국어"].index,inplace=True)

In [4]:
for index, row in ref.iterrows():
    ref['text'][index]=re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》;]','',ref['text'][index])

C:\Users\pthdu\anaconda3\envs\nltk\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
mecab=Mecab(dicpath="C:\mecab\mecab-ko-dic")
ref['token']=ref['text'].apply(mecab.pos)

In [6]:
#명사, 동사, 형용사, 어근, 부정사추출
def extraction(tokens):
    temp=""
    for word in tokens:
        if word[1] in['NNG','VV','VA','XR','VCN']:
            #print(word[1])
            temp+=str(word[0])+"/"+str(word[1])
            temp+=" "
    return temp

In [7]:
ref['token']=ref['token'].apply(extraction)
ref.head()
#type_list=list(set(ref['type']))

,space,type,language,day,text,score,token
0,힐스파,체험,한국어,201702,경치 최고 찜질방에서 하루 숙박했습니다 달맞이길 윗쪽에 위치해서 해운대 일대가 한눈...,5.0,경치/NNG 최고/NNG 찜질방/NNG 하루/NNG 숙박/NNG 달맞이/NNG 길/...
1,힐스파,체험,한국어,201711,경치가 좋은 스파 너무 좋아요 해운대 바다가 보이는 스파라서 스파하는동안 기분이 좋...,5.0,경치/NNG 좋/VA 좋/VA 해운대/NNG 바다/NNG 보이/VV 하/VV 동안/...
2,힐스파,체험,한국어,201710,여독풀기 좋은 장소 하루 종일 여행 후 피로를 녹이기 좋은 장소 주차도 잘 됩니다 ...,4.0,여독/NNG 풀/VV 좋/VA 장소/NNG 하루/NNG 종일/NNG 여행/NNG 후...
3,힐스파,체험,한국어,201602,달맞이 고개의 훌륭한 목욕탕 달맞이 고개에 자리하고 있으며 고지대에 위치한 덕분에 ...,5.0,달맞이/NNG 고개/NNG 훌륭/XR 목욕탕/NNG 달맞이/NNG 고개/NNG 자리...
4,힐스파,체험,한국어,201504,해운대 달맞이고개의 찜질방 찜질하는 찜질방입니다 스파도 같이 운영합니다 방문객이 많...,3.0,해운대/NNG 달맞이/NNG 고개/NNG 찜질방/NNG 찜질/NNG 찜질방/NNG ...


In [8]:
ref=ref.astype({'score':'float'})
ref=ref.astype({'score':'int'})
print(type(ref['score'][0]))

<class 'numpy.int32'>


In [9]:
for index,row in ref.iterrows():
    if row['score']>3:
        ref['score'][index]=1
    else:
        ref['score'][index]=0

C:\Users\pthdu\anaconda3\envs\nltk\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\pthdu\anaconda3\envs\nltk\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [10]:
#유형별 카테고리 추출
type_list=list(set(ref['type']))

In [11]:
# ref dataframe에 있는 token열 string화
category_df=[]
for index in range(len(type_list)):
    category_df.append(ref[ref['type']== type_list[index]])
    category_df[index]=category_df[index].astype({'token':'string'})

# CountVectorizer화

In [13]:
cv_count=[]
cv=[]

for i in range(len(type_list)):
    cv.append(CountVectorizer(
        min_df=1,
        ngram_range=(1,3),
        tokenizer=lambda x:x.split()))

    cv_count.append(cv[i].fit_transform(category_df[i]['token']))

In [14]:
cv_feature=[]
for i in range(len(type_list)):
    cv_feature.append(cv[i].get_feature_names())

# Logistic Regression

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid={'C':[0.001,0.01,0.1,1,10]}
grid=[]
for i in range(len(type_list)):
    grid.append(GridSearchCV(LogisticRegression(), param_grid, cv=5))
    grid[i].fit(cv_count[i],category_df[i]['score'])

C:\Users\pthdu\anaconda3\envs\nltk\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
C:\Users\pthdu\anaconda3\envs\nltk\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
C:\Users\pthdu\anaconda3\envs\nltk\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
AttributeError: 'str' object has no attribute 'decode'

  FitFailedWarning)
C:\Users\pthdu\anaconda3\envs\nltk\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
At

In [16]:
# Regression 돌린 후 토큰과 빈도수 값 붙여서 word_vec 담기
word_vec=[]
for i in range(len(type_list)):
    temp=grid[i].best_estimator_.coef_[0]
    feature=cv_feature[i]
    temp2=[]
    for j in range(len(temp)):
        temp2.append([temp[j],feature[j]])
    word_vec.append(temp2)

# 긍정과 부정으로 나눠서 빈도수 높은 순으로 정렬

In [18]:
import openpyxl

dic=[]
for i in range(len(type_list)):
    pos=[]
    neg=[]
    for j in range(len(word_vec[i])):
        if word_vec[i][j][0]>=0:
            pos.append(word_vec[i][j])
        else:
            neg.append(word_vec[i][j])
    pos.sort(reverse=True)
    neg.sort()
    dic_pos=pd.DataFrame(data=pos,columns=['pos_freq','pos'])
    dic_neg=pd.DataFrame(data=neg,columns=['neg_freq','neg'])
    dic.append(pd.concat([dic_pos,dic_neg],axis=1))
    


In [19]:
dic[3].head()

,pos_freq,pos,neg_freq,neg
0,0.647390,친구/nng,-0.881431,없/va
1,0.630453,최고/nng,-0.703005,같/va
2,0.582182,좋/va,-0.696637,카지노/nng
3,0.563713,훌륭/xr,-0.691472,생각/nng
4,0.549881,편안/nng,-0.615570,실망/nng


 # bokeh

In [20]:
from bokeh.layouts import gridplot,column,row
from bokeh.plotting import figure, output_file,show
from bokeh.models import ColumnDataSource,Select, DataTable, TableColumn
from bokeh.palettes import Spectral6
from bokeh.palettes import Spectral5
from bokeh.plotting import curdoc, figure
from bokeh.sampledata.autompg import autompg_clean as df
from bokeh.io import curdoc



In [21]:
#call back
def change(attrname, old, new):
    layout.children[1]=update()

    
gp=[]
def update():
    lis=[]
    t=ticker1.value
    t1=int(t)
    pos_token = list(dic[t1]['pos'][:20])
    pos_freq = list(dic[t1]['pos_freq'][:20])
    neg_token = list(dic[t1]['neg'][:20])
    neg_freq = list(dic[t1]['neg_freq'][:20])
    p = figure(x_range=pos_token, plot_width=1000,plot_height=300, title=type_list[t1]+"Postive")
    p.vbar(x=pos_token, top=pos_freq, width=0.7, color='darkblue')
    lis.append([p])
    p = figure(x_range=neg_token, plot_width=1000,plot_height=300, y_range=(-0.5,0.5), title=type_list[t1]+"Negative")
    p.vbar(x=neg_token, top=neg_freq, width=0.7, color='red')
    lis.append([p])
    gp=gridplot(lis)
    
    return gp
    


In [23]:
#menu값을 문자형숫자로 설정 해놓은 이유는 dropdown기능을 사용해 값을 선택하면 on_change함수가 실행되고
#chage()메서드에서 자식을 생성하는데 update()메서드도 실행됨.update()메서드에서 해당 값을 인트형으로 바꿔서 해당 인덱스의 그래프를 보여줌
menu = ["0","1","2","3","4","5","6","7","8","9"]

ticker1=Select(value="1",options=menu)#기본값 1로 설정
ticker1.on_change('value',change) #on_chage함수실행

layout=row(ticker1,update()) #ticker1이라는 dropdown객체를 생성했으면 html화면에 붙여 주어야 하는데 [드롭다운-그래프]순으로 보여주기 위해 row 방향으로 lqout생성
curdoc().add_root(layout) # layout을 root에 붙여줌
curdoc().title="LogisticRegression" #탭 창 이름 설정

In [25]:
show(layout) # 이렇게 하면 정적인 화면 밖에 보지 못함
#위에서 설정한 드롭다운 화면을 볼려면 
#bokeh를 설치하고
#anaconda의 가상환경에 이동한 후
#이 코드가 있는 상위폴더까지 이동한 후
#bokeh serve --show 파일이름.(파일타입)을 쳐주면 시간이 쫌 걸리나 실행이 됨
#몇번 에러가 떠서 실패할 수 있는데 와이파이 환경이 약하거나 끊기면 바로 error떠서 여러번 실행시켜봐야함.


You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



print(type_list)

# Bokeh가 아닌 img로 저장 할 경우

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import mglearn
import matplotlib.font_manager as fm

font_location=r'C:\Windows\Fonts\H2PORM.TTF'
font_name=fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font',family=font_name)
for i in range(len(typelist)):
    mglearn.tools.visualize_coefficients(grid[i].best_estimator_.coef_,cv_feature[i], n_top_features=20)
    #plt.show()
    plt.savefig(typelist[i]+'_keyword_7.jpg',facecolor='#eeeeee')